# Análisis exploratorio

In [1]:
import json
import os
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization, Activation
from keras.optimizers import Adam
from keras.constraints import max_norm
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle
import traceback
from sklearn.model_selection import GridSearchCV
from tqdm import tqdm
import numpy as np
from sklearn.preprocessing import OneHotEncoder

### Cargamos los registros

In [2]:
# Lista para almacenar los datos de todos los archivos
data = []

try:
    # Obtener la lista de archivos en la carpeta preprocessed
    carpeta_processed = '../Data/Processed/'
    archivos_processed = os.listdir(carpeta_processed)
    
    # Iterar sobre cada archivo
    for archivo in tqdm(archivos_processed, desc="Procesando archivos"):
        # Construir la ruta completa del archivo
        ruta_archivo = os.path.join(carpeta_processed, archivo)
        
        # Leer el contenido del archivo JSON
        with open(ruta_archivo, 'r') as f:
            datos_archivo = json.load(f)
        
        # Agregar los datos del archivo a la lista de datos totales
        data.append(datos_archivo)
        
    print(f"Datos cargados exitosamente desde la carpeta processed.")
    
except Exception as e:
    print(f"Error al cargar los datos desde {ruta_archivo}.")
    traceback.print_exc()

Procesando archivos: 100%|██████████████████████████████████████████████████████████████████| 240/240 [01:09<00:00,  3.47it/s]

Datos cargados exitosamente desde la carpeta processed.


## Preparación de datos

Para preparar nuestros datos, primero debemos estructurarlos en secuencias que representen las diversas actividades realizadas durante las sesiones de EEG. Cada paciente sigue un protocolo predefinido, ejecutando una serie de tareas en un orden específico que abarca desde la línea de base hasta acciones motoras e imaginadas. Este orden se presenta de la siguiente manera:

1. Baseline, eyes open
2. Baseline, eyes closed
3. Task 1 (open and close left or right fist)
4. Task 2 (imagine opening and closing left or right fist)
5. Task 3 (open and close both fists or both feet)
6. Task 4 (imagine opening and closing both fists or both feet)
7. Task 1
8. Task 2
9. Task 3
10. Task 4
11. Task 1
12. Task 2
13. Task 3
14. Task 4

In [3]:
def generar_secuencia(x):
    secuencia = [0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3]
    pacientes = x// 12
    res = []
    for e in range(pacientes):
        res.extend(secuencia)
    return res

In [4]:
data_clasificacion = generar_secuencia(len(data))
print(data_clasificacion)
print(f"Total registros: {len(data_clasificacion)}")
print(f"Total registros: {len(data)}")
print(f"Total de registros por canal: {len(data[0][0])}")

[0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3]
Total registros: 240
Total registros: 240
Total de registros por canal: 9600


Ejemplo de datos para prueba:

### Convertimos los datos de float 64 bits a float 32 bits.

In [5]:
# Convertir a numpy arrays
#eeg_data = np.array(data).astype(np.float32)
eeg_data = np.array(data, dtype=np.float32)

In [6]:
actividades = np.array(data_clasificacion)

In [7]:
# Dividir datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(eeg_data, actividades, test_size=0.2, random_state=42)

## Creamos el modelo CNN

## Modelo TCN

Se define un modelo CNN para la clasificación de las actividades EEG. La arquitectura incluye varias capas convolucionales (Conv1D), capas de pooling (MaxPooling1D), y capas densas. Se utilizan técnicas como la normalización por lotes (BatchNormalization) y el dropout para evitar el sobreajuste.

In [8]:
y_train_one_hot = to_categorical(y_train, num_classes=4)
y_test_one_hot = to_categorical(y_test, num_classes=4)

In [9]:
def create_model(dropout, conv_layers, filters, kernel_size, dense_layers):
    if conv_layers == 1 : 
        model = Sequential([
            Dropout(dropout, input_shape=(len(eeg_data[0]), len(eeg_data[0][0]))),
            Conv1D(filters=filters, kernel_size=kernel_size),
            BatchNormalization(),
            Activation("relu"),
            MaxPooling1D(2),
            Dropout(dropout),
            Conv1D(filters=filters, kernel_size=kernel_size),
            BatchNormalization(),
            Activation("relu"),
            MaxPooling1D(2),
            Flatten(),
            Dropout(dropout),
            Dense(dense_layers, activation='relu'),
            Dense(4, activation='softmax')
        ])
    else:
        model = Sequential([
            Dropout(dropout, input_shape=(len(eeg_data[0]), len(eeg_data[0][0]))),
            Conv1D(filters=filters, kernel_size=kernel_size),
            Conv1D(filters=filters, kernel_size=kernel_size),
            BatchNormalization(),
            Activation("relu"),
            MaxPooling1D(2),
            Dropout(dropout),
            Conv1D(filters=filters, kernel_size=kernel_size),
            Conv1D(filters=filters, kernel_size=kernel_size),
            BatchNormalization(),
            Activation("relu"),
            MaxPooling1D(2),
            Flatten(),
            Dropout(dropout),
            Dense(dense_layers, activation='relu'),
            Dense(4, activation='softmax')
        ])
    
    # Compile the model
    model.compile(
        loss="categorical_crossentropy",
        optimizer=Adam(learning_rate=0.0001),
        metrics=["accuracy"]
    )
    
    return model

param_grid = {
    'dropout': [0.9, 0.8, 0.7],
    'conv_layers': [2, 1],
    'filters': [128, 96, 64],
    'kernel_size': [10, 5, 3],
    'dense_layers': [128, 96, 64]
}

best_score = 0
score_history = []
best_params = {}

#total_iterations = len(param_grid['dropout']) * len(param_grid['conv_layers']) * len(param_grid['filters']) * len(param_grid['dense_layers']) * len(param_grid['kernel_size'])
total_iterations = np.prod([len(param_grid[key]) for key in param_grid])
count=0

for dropout in param_grid['dropout']:
    for conv_layers in param_grid['conv_layers']:
        for filters in param_grid['filters']:
            for kernel_size in param_grid['kernel_size']:
                for dense_layers in param_grid['dense_layers']:
                    # Creamos el modelo
                    model = create_model(dropout, conv_layers, filters, kernel_size, dense_layers)
                    history = model.fit(X_train, y_train_one_hot, epochs=60, validation_data=(X_test, y_test_one_hot), verbose=0)
                    score = history.history['val_accuracy'][-1]
                    
                    count = count + 1
                    print(f"Progreso: {count} / {total_iterations}")
                    if score > best_score:
                        best_score = score
                        score_history.append(score)
                        params = {'dropout': dropout, 'conv_layers': conv_layers, 
                              'filters': filters, 'kernel_size': kernel_size, 'dense_layers': dense_layers}
                        best_params = params
                        print(f"###################################################################################################")
                        print(f"Accuracy: {score}")
                        print(f"Params: {params}")
                        print(f"###################################################################################################")

print(f"###################################################################################################")
print("Best accuracy: {:.2f}% using {}".format(best_score * 100, best_params))
print(f"###################################################################################################")


C:\Users\Fernando\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\regularization\dropout.py:42: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Progreso: 1 / 162
###################################################################################################
Accuracy: 0.25
Params: {'dropout': 0.9, 'conv_layers': 2, 'filters': 128, 'kernel_size': 10, 'dense_layers': 128}
###################################################################################################
Progreso: 2 / 162
###################################################################################################
Accuracy: 0.3125
Params: {'dropout': 0.9, 'conv_layers': 2, 'filters': 128, 'kernel_size': 10, 'dense_layers': 96}
###################################################################################################
Progreso: 3 / 162
###################################################################################################
Accuracy: 0.3541666567325592
Params: {'dropout': 0.9, 'conv_layers': 2, 'filters': 128, 'kernel_size': 10, 'dense_layers': 64}
##########################################################################################

KeyboardInterrupt: 